In [ ]:
# ! pip3 install kfp==1.8.10
# ! pip3 install tfx==1.5.0
# ! pip3 install pyparsing==2.4.2

In [1]:
import tensorflow as tf
import tfx
import os

In [2]:
GCP_REGION = 'us-central1'
ARTIFACT_STORE_URI = 'gs://text-analysis-323506-artifact-store'
ENDPOINT = 'https://1b7bb986cc987470-dot-us-central1.pipelines.googleusercontent.com'
CUSTOM_SERVICE_ACCOUNT = 'my-api-sa@text-analysis-323506.iam.gserviceaccount.com'

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

In [3]:
os.environ['GCP_REGION'] = GCP_REGION
os.environ['ARTIFACT_STORE_URI'] = ARTIFACT_STORE_URI
os.environ['CUSTOM_SERVICE_ACCOUNT'] = CUSTOM_SERVICE_ACCOUNT
os.environ['PROJECT_ID'] = PROJECT_ID

In [4]:
PIPELINE_NAME = 'tfx_covertype_training'
MODEL_NAME = 'tfx_covertype_classifier'
DATA_ROOT_URI = 'gs://text-analysis-323506/covertype'
CUSTOM_TFX_IMAGE = 'gcr.io/{}/{}'.format(PROJECT_ID, PIPELINE_NAME)
RUNTIME_VERSION = '2.6'
PYTHON_VERSION = '3.7'
USE_KFP_SA=False
ENABLE_TUNING=False

In [5]:
os.environ['PIPELINE_NAME'] = PIPELINE_NAME
os.environ['MODEL_NAME'] = MODEL_NAME
os.environ['DATA_ROOT_URI'] = DATA_ROOT_URI
os.environ['CUSTOM_TFX_IMAGE'] = CUSTOM_TFX_IMAGE
os.environ['RUNTIME_VERSION'] = RUNTIME_VERSION
os.environ['PYTHON_VERSION'] = PYTHON_VERSION
os.environ['USE_KFP_SA'] = str(USE_KFP_SA)
os.environ['ENABLE_TUNING'] = str(ENABLE_TUNING)

In [6]:
%cd pipeline

/home/jupyter/TFX-Pipelines-on-Google-Cloud/pipeline


In [7]:
! tfx pipeline compile --engine kubeflow --pipeline_path runner.py

CLI
Compiling pipeline
running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying pipeline.py -> build/lib
copying runner.py -> build/lib
copying preprocessing.py -> build/lib
copying features.py -> build/lib
copying config.py -> build/lib
copying model.py -> build/lib
/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,
installing to /tmp/tmpczv6m_gz
running install
running install_lib
copying build/lib/preprocessing.py -> /tmp/tmpczv6m_gz
copying build/lib/features.py -> /tmp/tmpczv6m_gz
copying build/lib/pipeline.py -> /tmp/tmpczv6m_gz
copying build/lib/config.py -> /tmp/tmpczv6m_gz
copying build/lib/model.py -> /tmp/tmpczv6m_gz
copying build/lib/runner.py -> /tmp/tmpczv6m_gz
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_c